<a href="https://colab.research.google.com/github/artdreamer/battery_early_capacity_trajectory_prediction/blob/main/Elastic_deltaQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

P. M. Attia, K. A. Severson, and J. D. Witmer, “Statistical learning for accurate and interpretable battery lifetime prediction,” arXiv Prepr. arXiv2101.01885, 2021.

This note book is created based on the author's source notebook at 10.5281/zenodo.4420638
The results generated from this notebook are almost identical to the source notebook.

In [21]:
work_path = 'dataset/169 LFP'
import sys
sys.path.append(work_path)

In [41]:
import glob
import os
import numpy as np
from sklearn import preprocessing
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

Define `alphas` and `l1_ratios` to use for `RidgeCV` and `ElasticNetCV` hyperparameter optimization:

In [42]:
l1_ratios = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
alphas = np.logspace(0.001, 100, 20)

Define colors for train, test1, and test2 to roughly match Severson:

In [44]:
colors_list = ['Blues', 'Reds', 'Oranges']

In [45]:
np.random.seed(7)

## Load data
The data is stored in the `data` directory. The data was generated by the `generate_voltage_array.m` MATLAB script, which creates three folders of "voltage arrays" for the train, test1, and test2 splits in Severson et al.

We will load each dataset as 3D arrays of `[cell idx, voltage position, cycle number]`.

Note that because cycle 1 is unavailable for one batch, position 0 in axis 2 is cycle 2.
Similarly, cycle 100 is position 98 in axis 2.
In describing the cycle numbers in the text, I assume the first cycle is cycle 1 (i.e. references to cycle number in the text are one-indexed, not zero-indexed).


In [46]:
def sortKeyFunc(s):
    return int(os.path.basename(s)[4:-4])

def load_dataset(folder):
    files = glob.glob(work_path+f'/V_Q_curve/{folder}/*.csv')
    files.sort(key=sortKeyFunc) # glob returns list with arbitrary order
    
    l = len(files)
    dataset = np.zeros((l, 1000, 149))
    
    for k, file in enumerate(files):
        cell = np.genfromtxt(file, delimiter=',')
        dataset[k,:,:] = cell # flip voltage dimension
    
    return dataset


Load four datasets (`test1` = primary test set, `test2` = secondary test set, `test3` = tertiary test set):

In [47]:
data_train = load_dataset('train')
data_test1 = load_dataset('test1')
data_test2 = load_dataset('test2')
data_test3 = load_dataset('test3')

In [48]:
dataCycleLivesTrain = np.genfromtxt(work_path+f'/cycle_lives/train_cycle_lives.csv', delimiter=',')
dataCycleLivesTest1 = np.genfromtxt(work_path+f'/cycle_lives/test1_cycle_lives.csv', delimiter=',')
dataCycleLivesTest2 = np.genfromtxt(work_path+f'/cycle_lives/test2_cycle_lives.csv', delimiter=',')
dataCycleLivesTest3 = np.genfromtxt(work_path+f'/cycle_lives/test3_cycle_lives.csv', delimiter=',')

In [51]:
from scipy.io import loadmat
work_path
inputData = loadmat(work_path+'/Multi_variable_model_input_x_cycle_100.mat')
outputData = loadmat(work_path+'/Cycle_output_6_28_100_points.mat')

cellsTrain = np.squeeze(outputData['cells_train'])-1
cellsTest1 = np.squeeze(outputData['cells_test1'])-1
cellsTest2 = np.squeeze(outputData['cells_test2'])-1
cellsTest3 = np.squeeze(outputData['cells_test3'])-1

In [52]:
cycle_lives_data = loadmat(work_path+'/cycle_lives_different_capacity_levels.mat')
# cyclelives_80_elasticnet_discharge = cycle_lives_data['cycle_lives']
# cyclelives_85_elasticnet_discharge = cycle_lives_data['cycle_lives_85']
# cyclelives_90_elasticnet_discharge = cycle_lives_data['cycle_lives_90']

In [53]:
# dataCycleLivesTrain = cyclelives_90_elasticnet_discharge[cellsTrain]
# dataCycleLivesTest1 = cyclelives_90_elasticnet_discharge[cellsTest1]
# dataCycleLivesTest2 = cyclelives_90_elasticnet_discharge[cellsTest2]
# dataCycleLivesTest3 = cyclelives_90_elasticnet_discharge[cellsTest3]

Confirm `124 -1(outlier) + 45 =  168` cells are loaded:

In [54]:
print(len(data_train) + len(data_test1) + len(data_test2) + len(data_test3))
print(len(dataCycleLivesTrain) + len(dataCycleLivesTest1) + len(dataCycleLivesTest2) + len(dataCycleLivesTest3))

168
168


Define `y` values as log10 of cycle life:

In [33]:
y_train = np.log10(dataCycleLivesTrain)
y_test1 = np.log10(dataCycleLivesTest1)
y_test2 = np.log10(dataCycleLivesTest2)
y_test3 = np.log10(dataCycleLivesTest3)

Get median cycle lives for each dataset:

In [34]:
print(np.median(dataCycleLivesTrain))
print(np.median(dataCycleLivesTest1))
print(np.median(dataCycleLivesTest2))

527.0
580.0
964.5


Create helper function for reducing RMSE and MAPE boilerplate:

In [35]:
def get_RMSE_for_all_datasets(y_train_pred, y_test1_pred, y_test2_pred, y_test3_pred):
    """
    Calculate RMSE for three datasets. Use units of cycles instead of log(cycles)
    """
    
    RMSE_train = mean_squared_error(np.power(10, y_train), np.power(10, y_train_pred), squared=False)
    RMSE_test1 = mean_squared_error(np.power(10, y_test1), np.power(10, y_test1_pred), squared=False)
    RMSE_test2 = mean_squared_error(np.power(10, y_test2), np.power(10, y_test2_pred), squared=False)
    RMSE_test3 = mean_squared_error(np.power(10, y_test3), np.power(10, y_test3_pred), squared=False)

    return RMSE_train, RMSE_test1, RMSE_test2, RMSE_test3

def get_MAE_for_all_datasets(y_train_pred, y_test1_pred, y_test2_pred, y_test3_pred):
    """
    Calculate RMSE for three datasets. Use units of cycles instead of log(cycles)
    """
    
    MAE_train = mean_absolute_error(np.power(10, y_train), np.power(10, y_train_pred))
    MAE_test1 = mean_absolute_error(np.power(10, y_test1), np.power(10, y_test1_pred))
    MAE_test2 = mean_absolute_error(np.power(10, y_test2), np.power(10, y_test2_pred))
    MAE_test3 = mean_absolute_error(np.power(10, y_test3), np.power(10, y_test3_pred))

    return MAE_train, MAE_test1, MAE_test2, MAE_test3

def get_MAPE_for_all_datasets(y_train_pred, y_test1_pred, y_test2_pred, y_test3_pred):
    """
    Calculate RMSE for three datasets. Use units of cycles instead of log(cycles)
    """
    
    MAPE_train = mean_absolute_percentage_error(np.power(10, y_train), np.power(10, y_train_pred))
    MAPE_test1 = mean_absolute_percentage_error(np.power(10, y_test1), np.power(10, y_test1_pred))
    MAPE_test2 = mean_absolute_percentage_error(np.power(10, y_test2), np.power(10, y_test2_pred))
    MAPE_test3 = mean_absolute_percentage_error(np.power(10, y_test3), np.power(10, y_test3_pred))

    return MAPE_train, MAPE_test1, MAPE_test2, MAPE_test3

## Train dummy model
Use mean of training set:

In [36]:
dummy_regr = DummyRegressor()
dummy_regr.fit(data_train, y_train)
y_pred_train_dummy = dummy_regr.predict(data_train)
y_pred_test1_dummy = dummy_regr.predict(data_test1)
y_pred_test2_dummy = dummy_regr.predict(data_test2)
y_pred_test3_dummy = dummy_regr.predict(data_test3)

# get_RMSE_for_all_datasets(y_pred_train_dummy, y_pred_test1_dummy, y_pred_test2_dummy, y_pred_test3_dummy)
get_MAPE_for_all_datasets(y_pred_train_dummy, y_pred_test1_dummy, y_pred_test2_dummy, y_pred_test3_dummy)

(0.29629703002891594,
 0.28201620101156005,
 0.36053894461067254,
 0.23099171476250335)


## Learning from the raw features in $ \Delta Q_{100-10}(V) $

In Severson et al. and previous sections, we used summary statistics of $ \Delta Q_{100-10}(V) $ as features for training. In this section, we consider using the raw Q-V pairs as the features instead of the summary statistics. The primary motivation is to aid interpretability by investigating the weights assigned to the various voltages, but this may also improve prediction.

### Define X matrices

Here, each feature is $ \Delta Q_{100-10}(V) $ for a range of voltages.



In [38]:
DeltaQ_100_minus_10_train = data_train[:, :, 98] - data_train[:, :, 8]
DeltaQ_100_minus_10_test1 = data_test1[:, :, 98] - data_test1[:, :, 8]
DeltaQ_100_minus_10_test2 = data_test2[:, :, 98] - data_test2[:, :, 8]
DeltaQ_100_minus_10_test3 = data_test3[:, :, 98] - data_test3[:, :, 8]

# Define sampling frequency
freq = 10

# Define X matrices
X_train = DeltaQ_100_minus_10_train[:,0::freq]
X_test1 = DeltaQ_100_minus_10_test1[:,0::freq]
X_test2 = DeltaQ_100_minus_10_test2[:,0::freq]
X_test3 = DeltaQ_100_minus_10_test3[:,0::freq]

# Scale via standarization
scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test1_scaled = scaler.transform(X_test1)
X_test2_scaled = scaler.transform(X_test2)
X_test3_scaled = scaler.transform(X_test3)

model = ElasticNetCV(l1_ratio=l1_ratios, cv=5, random_state=0, max_iter=100000)
model.fit(X_train_scaled, y_train)

ElasticNetCV(cv=5, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
             max_iter=100000, random_state=0)

In [39]:
y_train_pred = model.predict(X_train_scaled).squeeze()
y_test1_pred = model.predict(X_test1_scaled).squeeze()
y_test2_pred = model.predict(X_test2_scaled).squeeze()
y_test3_pred = model.predict(X_test3_scaled).squeeze()
RMSE_train, RMSE_test1, RMSE_test2, RMSE_test3 = get_RMSE_for_all_datasets(y_train_pred, y_test1_pred, y_test2_pred, y_test3_pred)
MAE_train, MAE_test1, MAE_test2, MAE_test3 = get_MAE_for_all_datasets(y_train_pred, y_test1_pred, y_test2_pred, y_test3_pred)
MAPE_train, MAPE_test1, MAPE_test2, MAPE_test3 = get_MAPE_for_all_datasets(y_train_pred, y_test1_pred, y_test2_pred, y_test3_pred)

In [40]:
print(RMSE_train, RMSE_test1, RMSE_test2, RMSE_test3)
print(MAE_train, MAE_test1, MAE_test2, MAE_test3)
print(MAPE_train, MAPE_test1, MAPE_test2, MAPE_test3)

92.49902853861661 131.57878076157422 196.14544964776036 294.5205285936985
60.41952236672021 84.01115792843709 135.61857937103937 271.48557908090595
0.08599977039112634 0.10626715270870253 0.1250413962363983 0.35306133221230823


In [ ]:
# 'cyclelives_elasticnet_discharge.mat', 
cycle_lives_train_es = np.power(10, y_train_pred)
cycle_lives_test1_es = np.power(10, y_test1_pred)
cycle_lives_test2_es =  np.power(10, y_test2_pred)
cycle_lives_test3_es =  np.power(10, y_test3_pred)
cycle_lives_es = {}
cycle_lives_es['cycle_lives_train_es'] = cycle_lives_train_es
cycle_lives_es['cycle_lives_test1_es'] = cycle_lives_test1_es
cycle_lives_es['cycle_lives_test2_es'] = cycle_lives_test2_es
cycle_lives_es['cycle_lives_test3_es'] = cycle_lives_test3_es
# import pickle
# f = open(work_path+'/data/cyclelives_elasticnet_deltaQ.pkl', 'wb')
# pickle.dump(cycle_lives_es, f)
# f.close()

In [ ]:
# import pickle
# f = open(work_path+'/data/cyclelives_90_elasticnet_deltaQ.pkl', 'wb')
# pickle.dump(cycle_lives_es, f)
# f.close()